## 2013 ~ 2016년도의 태양광 발전량 데이터 엑셀 파일과 2013 ~ 2016년도의 날씨 데이터 엑셀 파일을 활용하여 날씨에 따른 태양광 발전량 예측


In [ ]:
from google.colab import drive
drive.mount('/gdrive', force_remount=True)

Mounted at /gdrive


In [ ]:
import tensorflow as tf
print("TensorFlow version:", tf.__version__)



TensorFlow version: 2.12.0


In [ ]:
from tensorflow.keras.layers import Dense, Flatten, Conv2D
from tensorflow.keras import Model

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
#13년도 태양광 발전량을 가져옴
#해가 떠있어야 태양광 발전이 가능하므로 8시부터 18시까지의 데이터만 추출
sun13 = pd.read_excel('//content/sample_data/부산지역-태양광발전량.xlsx', index_col = 0)
energy13 = sun13[[8,9,10,11,12,13,14,15,16,17,18]]
energy13

,8,9,10,11,12,13,14,15,16,17,18
년월일,,,,,,,,,,,
2013-01-01,0.360,19.944,54.144,132.840,197.100,158.508,198.180,147.708,65.196,14.652,0.108
2013-01-02,0.360,20.736,87.084,159.948,247.284,265.644,234.396,160.884,77.148,14.400,0.036
2013-01-03,0.180,21.168,80.748,149.508,213.876,271.584,266.004,177.912,63.288,14.832,0.216
2013-01-04,0.252,20.592,96.552,160.200,259.704,286.020,265.824,185.292,93.780,18.576,0.072
2013-01-05,0.180,22.464,110.628,200.844,277.920,297.612,268.632,130.392,79.488,20.052,0.144
...,...,...,...,...,...,...,...,...,...,...,...
2013-12-27,0.396,19.260,78.912,160.416,233.784,267.048,246.024,167.868,80.856,13.680,0.000
2013-12-28,0.108,26.316,108.756,198.000,266.220,283.104,257.760,175.428,80.100,13.608,0.000
2013-12-29,0.216,22.896,95.292,175.608,251.388,270.828,248.616,167.328,83.664,14.040,0.000


In [ ]:
#날씨데이터가 시간별로 일렬로 나열되어 있기 때문에 발전량도 시간에 따라 일렬로 정렬해 주는 과정
output_df = pd.DataFrame(energy13.values.flatten(), columns=['발전량'])
output_df

,발전량
0,0.360
1,19.944
2,54.144
3,132.840
4,197.100
...,...
4010,215.532
4011,145.260
4012,65.160
4013,13.572


In [ ]:
#날씨 데이터를 가져옴
#발전량 데이터가 8시 부터 18시까지의 데이터이므로 날씨 데이터도 같은 시간대만 추출
past13 = pd.read_excel('/content/sample_data/past-2013.xlsx', index_col = 0)
#past13.loc[past13['Time'].str.contains('12')]
wt13 = past13[past13['Time'].astype(str).str.contains('08|09|10|11|12|13|14|15|16|17|18')]
wt13

,Time,Sunrise,Sunset,Temperature,Humidity,cloudCover,Visibility,uvIndex,windSpeed,Isolation
Date,,,,,,,,,,
2013-01-01,08:00:00,07:33:00,17:24:00,-6.36,0.88,None,None,None,1.02,0.02
2013-01-01,09:00:00,07:33:00,17:24:00,1.11,0.46,0.26,14.994,0,0.11,0.37
2013-01-01,10:00:00,07:33:00,17:24:00,1.07,0.68,0.44,14.994,1,2.06,0.65
2013-01-01,11:00:00,07:33:00,17:24:00,-0.35,0.71,0.44,10.003,2,1.55,1.20
2013-01-01,12:00:00,07:33:00,17:24:00,6.59,0.55,0.62,14.994,2,4.16,1.42
...,...,...,...,...,...,...,...,...,...,...
2013-12-31,14:00:00,07:33:00,17:23:00,9.63,0.38,0,8.998,2,6.18,1.63
2013-12-31,15:00:00,07:33:00,17:23:00,10.16,0.32,0,8.998,1,8.18,1.32
2013-12-31,16:00:00,07:33:00,17:23:00,8.64,0.38,None,8.998,None,5.66,0.77


In [ ]:
#날씨데이터와 발전량 데이터를 합친 데이터테이블을 만듦
wt13_1 = wt13.reset_index()
output_df_1 = output_df.reset_index()
result = pd.concat([wt13_1, output_df_1["발전량"]], axis=1)
result

,Date,Time,Sunrise,Sunset,Temperature,Humidity,cloudCover,Visibility,uvIndex,windSpeed,Isolation,발전량
0,2013-01-01,08:00:00,07:33:00,17:24:00,-6.36,0.88,None,None,None,1.02,0.02,0.360
1,2013-01-01,09:00:00,07:33:00,17:24:00,1.11,0.46,0.26,14.994,0,0.11,0.37,19.944
2,2013-01-01,10:00:00,07:33:00,17:24:00,1.07,0.68,0.44,14.994,1,2.06,0.65,54.144
3,2013-01-01,11:00:00,07:33:00,17:24:00,-0.35,0.71,0.44,10.003,2,1.55,1.20,132.840
4,2013-01-01,12:00:00,07:33:00,17:24:00,6.59,0.55,0.62,14.994,2,4.16,1.42,197.100
...,...,...,...,...,...,...,...,...,...,...,...,...
4010,2013-12-31,14:00:00,07:33:00,17:23:00,9.63,0.38,0,8.998,2,6.18,1.63,215.532
4011,2013-12-31,15:00:00,07:33:00,17:23:00,10.16,0.32,0,8.998,1,8.18,1.32,145.260
4012,2013-12-31,16:00:00,07:33:00,17:23:00,8.64,0.38,None,8.998,None,5.66,0.77,65.160
4013,2013-12-31,17:00:00,07:33:00,17:23:00,8.64,0.41,None,9.988,None,5.15,0.28,13.572


In [ ]:
result

,Date,Time,Sunrise,Sunset,Temperature,Humidity,cloudCover,Visibility,uvIndex,windSpeed
0,2013-01-01,08:00:00,07:33:00,17:24:00,-6.36,0.88,None,None,None,1.02
1,2013-01-01,09:00:00,07:33:00,17:24:00,1.11,0.46,0.26,14.994,0,0.11
2,2013-01-01,10:00:00,07:33:00,17:24:00,1.07,0.68,0.44,14.994,1,2.06
3,2013-01-01,11:00:00,07:33:00,17:24:00,-0.35,0.71,0.44,10.003,2,1.55
4,2013-01-01,12:00:00,07:33:00,17:24:00,6.59,0.55,0.62,14.994,2,4.16
...,...,...,...,...,...,...,...,...,...,...
4010,2013-12-31,14:00:00,07:33:00,17:23:00,9.63,0.38,0,8.998,2,6.18
4011,2013-12-31,15:00:00,07:33:00,17:23:00,10.16,0.32,0,8.998,1,8.18
4012,2013-12-31,16:00:00,07:33:00,17:23:00,8.64,0.38,None,8.998,None,5.66
4013,2013-12-31,17:00:00,07:33:00,17:23:00,8.64,0.41,None,9.988,None,5.15


In [ ]:
result.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4003 entries, 0 to 4014
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   Date         4003 non-null   datetime64[ns]
 1   Time         4003 non-null   object        
 2   Sunrise      4003 non-null   object        
 3   Sunset       4003 non-null   object        
 4   Temperature  4003 non-null   float64       
 5   Humidity     4003 non-null   float64       
 6   cloudCover   4003 non-null   object        
 7   Visibility   4003 non-null   object        
 8   uvIndex      4003 non-null   object        
 9   windSpeed    4003 non-null   object        
 10  Isolation    4003 non-null   float64       
 11  발전량          4003 non-null   float64       
dtypes: datetime64[ns](1), float64(4), object(7)
memory usage: 406.6+ KB


In [ ]:
result13 = result
result13.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4003 entries, 0 to 4014
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   Date         4003 non-null   datetime64[ns]
 1   Time         4003 non-null   object        
 2   Sunrise      4003 non-null   object        
 3   Sunset       4003 non-null   object        
 4   Temperature  4003 non-null   float64       
 5   Humidity     4003 non-null   float64       
 6   cloudCover   4003 non-null   object        
 7   Visibility   4003 non-null   object        
 8   uvIndex      4003 non-null   object        
 9   windSpeed    4003 non-null   object        
 10  Isolation    4003 non-null   float64       
 11  발전량          4003 non-null   float64       
dtypes: datetime64[ns](1), float64(4), object(7)
memory usage: 406.6+ KB


# 13년도 데이터로 회귀 분석

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

# 데이터 불러오기
data = result

# X와 y 생성
X = data[['Isolation']]  # Isolation 데이터
y = data['발전량']  # 발전량 데이터

# 훈련 데이터와 검증 데이터로 나누기
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.1, random_state=41)

# 회귀 모델 생성 및 학습
model = LinearRegression()
model.fit(X_train, y_train)

# 검증 데이터로 예측
predictions = model.predict(X_val)

# 정확도 계산
accuracy = r2_score(y_val, predictions)
print("모델 정확도: {:.2f}".format(accuracy))

모델 정확도: 0.79


In [ ]:
#14년도 날씨와 발전량 데이터를 앞선 13년도와 같은 방식으로 만듦
sun14 = pd.read_excel('//content/sample_data/부산지역-태양광발전량.xlsx', index_col = 0, sheet_name = 1)
energy14 = sun14[[8,9,10,11,12,13,14,15,16,17,18]]
output_df_14 = pd.DataFrame(energy14.values.flatten(), columns=['발전량'])
past14 = pd.read_excel('/content/sample_data/past-2014.xlsx', index_col = 0)
wt14 = past14[past14['Time'].astype(str).str.contains('08|09|10|11|12|13|14|15|16|17|18')]
wt14_1 = wt14.reset_index()
output_df_14_1 = output_df_14.reset_index()
result14 = pd.concat([wt14_1, output_df_14_1["발전량"]], axis=1)
result14

,Date,Time,Sunrise,Sunset,Temperature,Humidity,cloudCover,Visibility,uvIndex,windSpeed,Isolation,발전량
0,2014-01-01,08:00:00,07:33:00,17:24:00,1.63,0.61,None,8.998,None,1.55,0.02,0.180
1,2014-01-01,09:00:00,07:33:00,17:24:00,6.30,0.40,None,9.988,None,2.54,0.35,17.388
2,2014-01-01,10:00:00,07:33:00,17:24:00,7.64,0.44,None,9.988,None,0.52,0.95,77.472
3,2014-01-01,11:00:00,07:33:00,17:24:00,9.63,0.30,None,9.988,None,4.12,1.41,158.184
4,2014-01-01,12:00:00,07:33:00,17:24:00,9.63,0.24,None,9.988,None,4.63,1.71,226.188
...,...,...,...,...,...,...,...,...,...,...,...,...
4010,2014-12-31,14:00:00,07:33:00,17:23:00,4.63,0.72,0.75,8.005,2,5.09,1.06,33.730
4011,2014-12-31,15:00:00,07:33:00,17:23:00,5.56,0.56,0.26,9.638,1,6.70,1.06,52.270
4012,2014-12-31,16:00:00,07:33:00,17:23:00,4.84,0.55,0.19,9.638,0,5.94,0.76,30.820
4013,2014-12-31,17:00:00,07:33:00,17:23:00,4.63,0.39,0.19,10.003,0,6.21,0.34,8.170


In [ ]:
#더 큰 데이터로 분석하고자 13년도와 14년도 데이터를 합침
result = pd.concat([result13, result14])
result

,Date,Time,Sunrise,Sunset,Temperature,Humidity,cloudCover,Visibility,uvIndex,windSpeed,Isolation,발전량
0,2013-01-01,08:00:00,07:33:00,17:24:00,-6.36,0.88,None,None,None,1.02,0.02,0.360
1,2013-01-01,09:00:00,07:33:00,17:24:00,1.11,0.46,0.26,14.994,0,0.11,0.37,19.944
2,2013-01-01,10:00:00,07:33:00,17:24:00,1.07,0.68,0.44,14.994,1,2.06,0.65,54.144
3,2013-01-01,11:00:00,07:33:00,17:24:00,-0.35,0.71,0.44,10.003,2,1.55,1.20,132.840
4,2013-01-01,12:00:00,07:33:00,17:24:00,6.59,0.55,0.62,14.994,2,4.16,1.42,197.100
...,...,...,...,...,...,...,...,...,...,...,...,...
4010,2014-12-31,14:00:00,07:33:00,17:23:00,4.63,0.72,0.75,8.005,2,5.09,1.06,33.730
4011,2014-12-31,15:00:00,07:33:00,17:23:00,5.56,0.56,0.26,9.638,1,6.7,1.06,52.270
4012,2014-12-31,16:00:00,07:33:00,17:23:00,4.84,0.55,0.19,9.638,0,5.94,0.76,30.820
4013,2014-12-31,17:00:00,07:33:00,17:23:00,4.63,0.39,0.19,10.003,0,6.21,0.34,8.170


# 13 ~ 14년도 데이터로 회귀 분석

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

# 데이터 불러오기
data = result

# X와 y 생성
X = data[['Isolation']]  # Isolation 데이터
y = data['발전량']  # 발전량 데이터

# 훈련 데이터와 검증 데이터로 나누기
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.1, random_state=0)

# 회귀 모델 생성 및 학습
model = LinearRegression()
model.fit(X_train, y_train)

# 검증 데이터로 예측
predictions = model.predict(X_val)

# 정확도 계산
accuracy = r2_score(y_val, predictions)
print("모델 정확도: {:.2f}".format(accuracy))

모델 정확도: 0.62


In [ ]:
#15년도 데이터 생성
sun15 = pd.read_excel('//content/sample_data/부산지역-태양광발전량.xlsx', index_col = 0, sheet_name = 2)
energy15 = sun15[[8,9,10,11,12,13,14,15,16,17,18]]
output_df_15 = pd.DataFrame(energy15.values.flatten(), columns=['발전량'])
past15 = pd.read_excel('/content/sample_data/past-2015.xlsx', index_col = 0)
wt15 = past15[past15['Time'].astype(str).str.contains('08|09|10|11|12|13|14|15|16|17|18')]
wt15_1 = wt15.reset_index()
output_df_15_1 = output_df_15.reset_index()
result15 = pd.concat([wt15_1, output_df_15_1["발전량"]], axis=1)
result15

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4015 entries, 0 to 4014
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   Date         4015 non-null   datetime64[ns]
 1   Time         4015 non-null   object        
 2   Sunrise      4015 non-null   object        
 3   Sunset       4015 non-null   object        
 4   Temperature  4015 non-null   float64       
 5   Humidity     4015 non-null   object        
 6   cloudCover   4015 non-null   object        
 7   Visibility   4015 non-null   object        
 8   uvIndex      4015 non-null   object        
 9   windSpeed    4015 non-null   float64       
 10  Isolation    4015 non-null   float64       
 11  발전량          4015 non-null   float64       
dtypes: datetime64[ns](1), float64(4), object(7)
memory usage: 376.5+ KB


In [ ]:
#13 ~ 15년도 데이터
result = pd.concat([result, result15])
result

,Date,Time,Sunrise,Sunset,Temperature,Humidity,cloudCover,Visibility,uvIndex,windSpeed,Isolation,발전량
0,2013-01-01,08:00:00,07:33:00,17:24:00,-6.36,0.88,None,None,None,1.02,0.02,0.360
1,2013-01-01,09:00:00,07:33:00,17:24:00,1.11,0.46,0.26,14.994,0,0.11,0.37,19.944
2,2013-01-01,10:00:00,07:33:00,17:24:00,1.07,0.68,0.44,14.994,1,2.06,0.65,54.144
3,2013-01-01,11:00:00,07:33:00,17:24:00,-0.35,0.71,0.44,10.003,2,1.55,1.20,132.840
4,2013-01-01,12:00:00,07:33:00,17:24:00,6.59,0.55,0.62,14.994,2,4.16,1.42,197.100
...,...,...,...,...,...,...,...,...,...,...,...,...
4010,2015-12-31,14:00:00,07:33:00,17:22:00,8.64,0.55,0.19,10.003,2,7.2,1.58,110.304
4011,2015-12-31,15:00:00,07:33:00,17:22:00,10.23,0.38,0.19,10.003,1,2.69,1.26,73.008
4012,2015-12-31,16:00:00,07:33:00,17:22:00,8.64,0.51,0.19,10.003,0,5.09,0.79,33.048
4013,2015-12-31,17:00:00,07:33:00,17:22:00,6.63,0.54,0.19,10.003,0,3.6,0.28,6.804


# 13 ~ 15년도 데이터로 회귀 분석

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

# 데이터 불러오기
data = result

# X와 y 생성
X = data[['Isolation']]  # Isolation 데이터
y = data['발전량']  # 발전량 데이터

# 훈련 데이터와 검증 데이터로 나누기
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.9, random_state=None)

# 회귀 모델 생성 및 학습
model = LinearRegression()
model.fit(X_train, y_train)

# 검증 데이터로 예측
predictions = model.predict(X_val)

# 정확도 계산
accuracy = r2_score(y_val, predictions)
print("모델 정확도: {:.2f}".format(accuracy))

모델 정확도: 0.67


In [ ]:
#16년도 데이터 생성
sun16 = pd.read_excel('//content/sample_data/부산지역-태양광발전량.xlsx', index_col = 0, sheet_name = 3)
energy16 = sun16[[8,9,10,11,12,13,14,15,16,17,18]]
output_df_16 = pd.DataFrame(energy16.values.flatten(), columns=['발전량'])
past16 = pd.read_excel('/content/sample_data/past-2016.xlsx', index_col = 0)
wt16 = past16[past16['Time'].astype(str).str.contains('08|09|10|11|12|13|14|15|16|17|18')]
wt16_1 = wt16.reset_index()
output_df_16_1 = output_df_16.reset_index()
result16 = pd.concat([wt16_1, output_df_16_1["발전량"]], axis=1)
result16

,Date,Time,Sunrise,Sunset,Temperature,Humidity,cloudCover,Visibility,uvIndex,windSpeed,Isolation,발전량
0,2016-01-01,08:00:00,07:33:00,17:23:00,-2.36,0.82,None,10.003,None,2.60,0.04,0.000
1,2016-01-01,09:00:00,07:33:00,17:23:00,2.59,0.53,None,10.003,None,5.02,0.48,11.592
2,2016-01-01,10:00:00,07:33:00,17:23:00,1.80,0.65,None,10.003,None,4.37,1.15,51.444
3,2016-01-01,11:00:00,07:33:00,17:23:00,4.63,0.58,None,10.003,None,4.60,1.63,93.348
4,2016-01-01,12:00:00,07:33:00,17:23:00,8.40,0.37,0.38,10.003,2,2.22,1.85,128.268
...,...,...,...,...,...,...,...,...,...,...,...,...
4021,2016-12-31,14:00:00,07:33:00,17:23:00,8.64,0.51,None,10.003,None,5.09,1.68,120.816
4022,2016-12-31,15:00:00,07:33:00,17:23:00,9.75,0.43,0.01,10.003,1,3.62,1.32,83.304
4023,2016-12-31,16:00:00,07:33:00,17:23:00,8.64,0.59,0.19,10.003,0,3.10,0.81,39.996
4024,2016-12-31,17:00:00,07:33:00,17:23:00,8.64,0.55,0.19,10.003,0,3.10,0.32,8.532


In [ ]:
#13 ~ 16년도 데이터
result = pd.concat([result, result16])
result

,Date,Time,Sunrise,Sunset,Temperature,Humidity,cloudCover,Visibility,uvIndex,windSpeed,Isolation,발전량
0,2013-01-01 00:00:00,08:00:00,07:33:00,17:24:00,-6.36,0.88,None,None,None,1.02,0.02,0.360
1,2013-01-01 00:00:00,09:00:00,07:33:00,17:24:00,1.11,0.46,0.26,14.994,0,0.11,0.37,19.944
2,2013-01-01 00:00:00,10:00:00,07:33:00,17:24:00,1.07,0.68,0.44,14.994,1,2.06,0.65,54.144
3,2013-01-01 00:00:00,11:00:00,07:33:00,17:24:00,-0.35,0.71,0.44,10.003,2,1.55,1.20,132.840
4,2013-01-01 00:00:00,12:00:00,07:33:00,17:24:00,6.59,0.55,0.62,14.994,2,4.16,1.42,197.100
...,...,...,...,...,...,...,...,...,...,...,...,...
4021,2016-12-31,14:00:00,07:33:00,17:23:00,8.64,0.51,None,10.003,None,5.09,1.68,120.816
4022,2016-12-31,15:00:00,07:33:00,17:23:00,9.75,0.43,0.01,10.003,1,3.62,1.32,83.304
4023,2016-12-31,16:00:00,07:33:00,17:23:00,8.64,0.59,0.19,10.003,0,3.1,0.81,39.996
4024,2016-12-31,17:00:00,07:33:00,17:23:00,8.64,0.55,0.19,10.003,0,3.1,0.32,8.532


# 13 ~ 16년도 데이터로 회귀 분석

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

# 데이터 불러오기
data = result

# X와 y 생성
X = data[['Isolation']]  # Isolation 데이터
y = data['발전량']  # 발전량 데이터

# 훈련 데이터와 검증 데이터로 나누기
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=None)

# 회귀 모델 생성 및 학습
model = LinearRegression()
model.fit(X_train, y_train)

# 검증 데이터로 예측
predictions = model.predict(X_val)

# 정확도 계산
accuracy = r2_score(y_val, predictions)
print("모델 정확도: {:.2f}".format(accuracy))

모델 정확도: 0.64
